In [1]:
#将网页读入 BeautifulSoup并用html.parser剖析器防止警告的产生
import requests
from bs4 import BeautifulSoup
res=requests.get('http://news.sina.com.cn/china/')
res.encoding='utf-8'
soup=BeautifulSoup(res.text,'html.parser')

In [2]:
commenturl='http://comment5.news.sina.com.cn/page/info?version=1&\
format=json&channel=gn&newsid=comos-{}&group=undefined&\
compress=0&ie=utf-8&oe=utf-8&page=1&page_size=3&t_size=3&h_size=3'

In [3]:
#动态获取新闻的评论数
import re
import json
def getcomments(newsurl):
    m=re.search('doc-i(.*).shtml',newsurl)
    newsid=m.group(1)
    comments=requests.get(commenturl.format(newsid))
    jd = json.loads(comments.text.strip('var data='))
    return jd['result']['count']['total']

In [4]:
#新闻的信息抽取函数
def getNewsDetail(newsurl):
    result={}
    res=requests.get(newsurl)
    res.encoding='utf-8'
    soup=BeautifulSoup(res.text,'html.parser')
    result['tittle']=soup.select('.main-title')[0].text
    #result['newssource']=soup.select('.date-source a')[0].text
    timesource=soup.select('.date')[0].contents[0].strip()
    result['dt']=timesource
    #result['dt']=datetime.strptime(timesource,'%Y年%m月%d日%H:%M')
    result['article']=' '.join( [p.text.strip()  for p in soup.select('.article p')[:-1]])
    result['editor']=soup.select('.show_author')[0].text.lstrip('责任编辑：')
    result['comment']=getcomments(newsurl)
    return result

In [5]:
getNewsDetail('http://news.sina.com.cn/c/nd/2018-07-24/doc-ihftenhz6977167.shtml')

{'article': '原标题：外交部：睦邻友好是中菲关系的正确方向 据报道，23日，菲律宾总统杜特尔特发表国情咨文时提到菲中关系，回应了一段时间以来菲国内有关菲中关系的一些关切和质疑。杜高度评价中菲在打击跨国犯罪和禁毒等领域合作，并表示，中菲关系改善不代表菲放弃南海权益，双方正通过双边和多边渠道友好处理分歧。中国外交部发言人耿爽在今天（24日）的例行记者会上表示，中方注意到杜特尔特总统国情咨文中有关过去两年菲中关系持续改善，两国合作前所未有的积极评价。 耿爽表示，中方赞赏菲律宾政府在杜特尔特总统领导下坚持独立自主外交政策，在相互平等、相互尊重的基础上发展同各国的正常关系和互利合作。2016年中菲关系转圜以来，双方各领域合作全面恢复并取得显著进展，为两国人民带来了实实在在的利益。安全合作是两国一大合作重点。中方坚定支持杜特尔特总统禁毒、反恐和打击跨国犯罪的努力，助力菲政府维护和平安宁。近年来两国合作实践充分说明，睦邻友好是符合两国和两国人民利益的唯一选择，是我们应始终坚持的正确方向。 耿爽强调，杜特尔特总统就职以来，中菲就南海问题一直保持顺畅有效沟通，积极开展对话合作，维护了海上形势总体稳定。这一发展顺应两国人民的共同期盼，也为地区和平稳定作出重要贡献。中方愿同菲方一道，继续妥善管控分歧、聚焦务实合作，共同维护南海和平稳定。中方还将继续同包括菲律宾在内的东盟国家共建地区规则，推动“南海行为准则”磋商不断取得积极进展。（央视记者 申杨）',
 'comment': 109,
 'dt': '2018年07月24日 16:27',
 'editor': '霍宇昂 ',
 'tittle': '杜特尔特称中菲改善不代表放弃南海权益 中方回应'}

In [6]:
def parselistlinks(url):
    newsdetails=[]
    res=requests.get(url)
    jd=json.loads(res.text.lstrip("'  newsloadercallback(").rstrip(');'))
    for ent in jd['result']['data']:
        newsdetails.append(getNewsDetail(ent['url']))
    return newsdetails

In [7]:
url='http://api.roll.news.sina.com.cn/zt_list?channel=news&\
cat_1=gnxw&cat_2==gdxw1||=gatxw||=zs-pl||=mtjj&level==1||=2&\
show_ext=1&show_all=1&show_num=22&tag=1&format=json&page=2&\
callback=newsloadercallback&_=1532430031835'

In [45]:
parselistlinks(url)

[{'article': '原标题：台“友邦”洪都拉斯总统：中国影响力增加 是给中美洲所有人的机会 [环球网综合报道]台当局的所谓“邦交国”目前还有17个，其中包括中美洲国家洪都拉斯。就中美洲目前局势，洪都拉斯总统埃尔南德斯（Juan Orlando Hernandez）24日接受英国路透社专访时表示，欢迎中国在中美洲增加影响力。 据英国路透社9月24日报道，埃尔南德斯受访提到，美国曾承诺对中美洲国家洪都拉斯、危地马拉、萨尔瓦多加大投资，但自美国总统特朗普（Donald Trump）上任以来，援助日益减少。 至于中国，埃尔南德斯说对中国在中美洲不断增长的外交影响力（diplomatic presence）表示欢迎，并称这对当地所有人是一个机遇（an opportunity for all）。 中美洲国家巴拿马、萨尔瓦多分别于去年6月和今年8月与台当局“断交”并与中国大陆建交。 虽然认为中国不断增长的外交影响力令该地区国家“左右为难”（quandary），但埃尔南德斯也认为，其他国家很快就会跟进随萨尔瓦多和巴拿马，“我们发现中美洲已经发生变化，我认为美国对此有点后知后觉”。 洪都拉斯、萨尔瓦多、危地马拉同位于中美洲北三角。该地区常年政治动荡，是美国难民的最大来源地。 美国于2014年联合三国成立“中美洲北三角繁荣联盟计划”，旨在帮助发展经济、改善治安，减少三国赴美移民。不过美国的援助却持续减少，路透社称，2016年至2018年间，美国对上述三国援助总额下降了超过三分之一（1.95亿美元）。 目前，洪都拉斯仍是台当局17个“友邦”之一。至于否会转向中国大陆，路透社报道称，埃尔南德斯谨慎（careful）表示，目前仍与台湾在一起（still with），“仍押注在（betting on）与台湾保持商业关系，作为进入亚洲市场的窗口。” 台当局领导人上台以来，已有5个国家结束与台当局的“邦交”关系并与中国大陆建交，其中三个位于中美洲。目前仍有4个中美洲国家与台湾保持所谓“邦交”关系，洪都拉斯就是其中之一。 在台当局“邦交国”与台“断交”并与中国大陆建交一事上，中国外交部多次强调，世界上绝大多数国家都已作出同样的决定。事实一再证明，坚持一个中国原则是国际社会普遍共识，是人心所向，大势所趋。',
  'comment': 0,
  'dt': '2018年09月25日 19:20

In [46]:
url='http://api.roll.news.sina.com.cn/zt_list?channel=news&\
cat_1=gnxw&cat_2==gdxw1||=gatxw||=zs-pl||=mtjj&level==1||=2&\
show_ext=1&show_all=1&show_num=22&tag=1&format=json&page={}&\
callback=newsloadercallback&_=1532430031835'
news_total=[]
for i in range(1,2):
    newsurl=url.format(i)
    newsary=parselistlinks(newsurl)
    news_total.extend(newsary)

In [47]:
len(news_total) 

22

In [60]:
news_total

[{'article': '原标题：台“友邦”洪都拉斯总统：中国影响力增加 是给中美洲所有人的机会 [环球网综合报道]台当局的所谓“邦交国”目前还有17个，其中包括中美洲国家洪都拉斯。就中美洲目前局势，洪都拉斯总统埃尔南德斯（Juan Orlando Hernandez）24日接受英国路透社专访时表示，欢迎中国在中美洲增加影响力。 据英国路透社9月24日报道，埃尔南德斯受访提到，美国曾承诺对中美洲国家洪都拉斯、危地马拉、萨尔瓦多加大投资，但自美国总统特朗普（Donald Trump）上任以来，援助日益减少。 至于中国，埃尔南德斯说对中国在中美洲不断增长的外交影响力（diplomatic presence）表示欢迎，并称这对当地所有人是一个机遇（an opportunity for all）。 中美洲国家巴拿马、萨尔瓦多分别于去年6月和今年8月与台当局“断交”并与中国大陆建交。 虽然认为中国不断增长的外交影响力令该地区国家“左右为难”（quandary），但埃尔南德斯也认为，其他国家很快就会跟进随萨尔瓦多和巴拿马，“我们发现中美洲已经发生变化，我认为美国对此有点后知后觉”。 洪都拉斯、萨尔瓦多、危地马拉同位于中美洲北三角。该地区常年政治动荡，是美国难民的最大来源地。 美国于2014年联合三国成立“中美洲北三角繁荣联盟计划”，旨在帮助发展经济、改善治安，减少三国赴美移民。不过美国的援助却持续减少，路透社称，2016年至2018年间，美国对上述三国援助总额下降了超过三分之一（1.95亿美元）。 目前，洪都拉斯仍是台当局17个“友邦”之一。至于否会转向中国大陆，路透社报道称，埃尔南德斯谨慎（careful）表示，目前仍与台湾在一起（still with），“仍押注在（betting on）与台湾保持商业关系，作为进入亚洲市场的窗口。” 台当局领导人上台以来，已有5个国家结束与台当局的“邦交”关系并与中国大陆建交，其中三个位于中美洲。目前仍有4个中美洲国家与台湾保持所谓“邦交”关系，洪都拉斯就是其中之一。 在台当局“邦交国”与台“断交”并与中国大陆建交一事上，中国外交部多次强调，世界上绝大多数国家都已作出同样的决定。事实一再证明，坚持一个中国原则是国际社会普遍共识，是人心所向，大势所趋。',
  'comment': 0,
  'dt': '2018年09月25日 19:20

In [71]:
import pymysql
db = pymysql.connect('localhost','root','951010','student',charset='utf8')
cur = db.cursor()
sqlc = '''
    create table news(
    id int(11) not null auto_increment primary key,
    article text not null,
    comment int(5) not null,
    dt varchar(20) not null,
    editor varchar(50) not null,
    tittle varchar(50) not null
    )
    '''
cur.execute(sqlc)#创建数据库

for new_total in news_total:
    article = new_total['article']
    comment = new_total['comment']
    dt = new_total['dt']
    editor = new_total['editor']
    tittle = new_total['tittle']
    sqli = '''insert into news(article,comment,dt,editor,tittle)
    values('%s','%d','%s','%s','%s')
    '''%(article,comment,dt,editor,tittle)
    try:
        cur.execute(sqli)#执行sql语句 ，插入数据  
        db.commit()# 提交到数据库执行
        print('导入数据库成功')
    except:
        db.rollback()# 如果发生错误则回滚
        print('shibai')
    cursor.close()#关闭游标

导入数据库成功
导入数据库成功
导入数据库成功
导入数据库成功
导入数据库成功
导入数据库成功
导入数据库成功
导入数据库成功
导入数据库成功
导入数据库成功
导入数据库成功
导入数据库成功
导入数据库成功
导入数据库成功
导入数据库成功
导入数据库成功
导入数据库成功
导入数据库成功
导入数据库成功
导入数据库成功
导入数据库成功
导入数据库成功


In [11]:
import pandas
df = pandas.DataFrame(news_total)
#df.head(10)
df

,article,comment,dt,editor,tittle
0,原标题：英国官员曾想施压葡萄牙收紧政策，杜绝香港人“曲线入英” [文/观察者网 谷智轩]近日...,19,2018年07月24日 21:58,余鹏飞,拦住那些香港人 英解密文件要让一些人心寒了(图)
1,原标题：今天这消息对“台独”打击太大了！ 台湾“奥运正名”让台中失去东亚青会主办权，连日本都...,1527,2018年07月24日 22:37,霍宇昂,台中市被取消主办东亚青年运动会 连日本都没帮它
2,原标题：对抗后我军飞行员后怕：我们居然还敢喊“首战用我、用我必胜” 所有的实战化训练，如果不...,2567,2018年07月24日 21:48,霍宇昂,对抗后我军飞行员后怕:还敢喊首战用我 用我必胜?
3,原标题：别认错：港元5种新钞样式曝光！ 据香港东网今日（24日）报道，香港金融管理局24日中...,14,2018年07月24日 21:43,霍宇昂,别认错：港元5种新钞样式曝光(图)
4,原标题：外媒：美国航企将于最后一刻 改用城市名标注台湾 海外网7月24日电 中国民航局要求4...,8477,2018年07月24日 20:52,霍宇昂,外媒：美国航企将于最后一刻 改用城市名标注台湾
5,新华社北京7月24日电（记者岳东兴 公兵）记者24日了解到，近来引发关注的有关“国足踢中超”...,6,2018年07月24日 20:49,霍宇昂,新华社：有关“国足踢中超”的建议暂无定论
6,原标题：南昌大学性侵案最新：受害女生将老师学校告上法庭，网传学校威胁 来源：中国新闻周刊 校...,429,2018年07月24日 20:47,霍宇昂,南昌大学性侵案最新:受害女生将老师学校告上法庭
7,原标题：锐参考|看着解放军近50艘军舰离开的背影，台当局忽然回过神来 千军万马，惊涛骇浪。 ...,489,2018年07月24日 20:28,霍宇昂,看着解放军近50艘军舰离开背影 台当局忽然回过神
8,原标题：涉文物犯罪A级通缉令在逃人员韩万里在河南落网 法制网讯 记者刘子阳 7月24日，公安...,0,2018年07月24日 20:23,霍宇昂,涉文物犯罪A级通缉令在逃人员韩万里在河南落网
9,中新网7月24日电 据长春市公安局长春新区分局微信公众号消息，7月24日，长春长生生物科技有...,21898,2018年07月24日 20:05,霍宇昂,长春长生公司董事长等15人被刑拘


In [12]:
df.to_excel('news.xlsx')